# Local LLM Example w/ RAG on OnLogic K800

This notebook shows an example pipeline for locally running an LLM with RAG integration, using Ollama and LangChain. 

Ollama is a user friendlly, free, and open-source tool for local LLM execution. For this example, we pull 2 models from Ollama:
* Llama3.1:8b-instruct-fp16 - This is the general purpose model that the pipeline is built from. It works well on its own, but implementing a RAG allows for more control over responses and ensures the model adheres to reliable data sources. 
* nomic-embed-text - This is an Ollama supported text embedding model (137M parameters) that allows the pipeline to pull relevant context sources from the context database based on their similarity with the prompt.

To get started with Ollama, visit their website and download the app for your device .
After installing, run:

`ollama pull llama3.1:8b-instruct-fp16`

`ollama pull nomic-embed-text`

to download and prepare the models for local execution. Ollama has a wide catalog of llms of different sizes and specialties, so browse to find one that matches your usecase and hardware.  

Getting the models running is as easy as running

`ollama start llama3.1:8b`

Which allows you to chat with the LLM through the command line. Run `ollama ps` to see the models you have currently loaded in memory and `ollama list` to see your list of locally downloaded models. 

Langchain is the second tool used for this example, which is used to define the overall RAG pipeline. We first create an embeddings database using LangChain Chroma to store the embeddings for each context document, and then define a chain which collects the most relevent documents and provides them to the llama model which in turn responds to the question. This is a super simple example of the types of chains that LangChain supports, which can be much more complex and customizable, allowing for things like web search and model self-correction. 


All of the LangChain libraries used here can be installed by: 

`pip install langchain langchain_community`

`pip install -qU langchain_chroma`

`pip install -qU langchain_ollama`

`pip install -qU langchain_unstructured`

This notebook was run locally on an OnLogic K804 with one 20GB Nvidia A4000 GPU. 

This example script will walk through the creation of a simple chat bot for giving technical support for a couple of onlogic pcs. To do this, we need to provide it context from which it will create its answers. To do this, we'll pull a collection of product manuals from onlogic.com, and let langchain and ollama convert them into embeddings, which allows the RAG to gain some semantic meaning of the content in each document. To do this, we'll use langchain's unstructured library to parse the documents and separate them into sections. This requires some additional installation, starting with 

`pip install -qU "unstructured[pdf]"`

When parsing pdfs locally, the unstructured library also requires some additional dependencies, Tessaract and Poppler. 

You can also do pdf parsing through the free unstructured API, which requires an API key. 

In [1]:
# Parse pdfs, creating a Document object for each section within each pdf
# this can take a while, depending on the number and size of the documents. 
from langchain_unstructured import UnstructuredLoader
from langchain_community.vectorstores.utils import filter_complex_metadata
from glob import glob

docs = []

pdf_filepaths = glob('product_manuals/*.pdf')

for file in pdf_filepaths:
    loader = UnstructuredLoader(
        file_path=file,
        strategy="fast",
        chunking_strategy='by_title'
    )

    for doc in loader.lazy_load():
        docs.append(doc)
    
filter_complex_metadata(docs)

print(f'Total number of documents: {len(docs)}')
docs[0]

C:\Users\no\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: pikepdf C++ to Python logger bridge initialized


Total number of documents: 290


Document(metadata={'source': 'product_manuals\\OnLogic-FR201-Product-Manual.pdf', 'file_directory': 'product_manuals', 'filename': 'OnLogic-FR201-Product-Manual.pdf', 'last_modified': '2024-12-04T12:09:29', 'page_number': 1, 'orig_elements': 'eJzlmE1v3DYQhv8KoXOo8vsjt6aN2xRNY8TOyQ4MihzZAnalhVbrjwb97x1RMrCxt8D2oEO3x5f7ckXx0QxnePWtgBWsoR1umlS8JYWK0kVhHa0rnagyUtMgDKdcV44pGStWV8UbUqxhCCkMAed8K2LX9alpwwDbrFfhqdsNN3fQ3N4NOOKUwCnz6EOThjsc1N7g4KZr2mGcdXVlRcneEK5t6b++Ic/SyUlKoUp7QGc7DhTbp+0A6/EdzptHWF1sQoTiL/yhblZwk5oe4tD1T6Nh03dpF4ebdWh3YbUtZlMb1jD+/Kn9vbttIj37LBin55OZfszmcpPqIr9Me7sLt/mFrwpob4uveXSL/9qlpm4gb6dgQlEuKFOXXLxl/q3w4+wNzrxpd+sKenTxcZUDPI5bVeSHkvmhZHroOGV42uTFXTbDKr/XK3LGuJpxRoPmjiqoAnXgNHV1srwONnkvFyVnbWn2wDE5SaFNaV7r7P5vcxP73D7DfbNtupb82mzzeo9hBsyC9zFRZyOnigVGXbCBKmGDcwZYpRaLNluijUtXugxtkmqWXPmSH9DZ/n+nZqRT4B2jqRKGqqQDDSAcFVxhqkwRKi2WpWZEyfeo2VkKzg7qbD8hamdNvx1Ij1TCFkhXkylpro9PlrXTpgYWqYAx8IKReMwlTrULXEeNgSgXS5YTM6dLuYfQz1IwVsoDOttPCOGXDW4rkHddN5DfdusN9OS8aXHjjsIXRAjAlKJJ+EhVHST1x

In [2]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

# Use a local embedding model to store the embddings of all context documents in a vector database
local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(docs, local_embeddings)

# formatting function used by the Llama model to "read" the context documents.
def format_docs(docs):
    return "\n\n".join(f'context source: {doc.metadata["filename"]}\nContent:\n{doc.page_content}' for doc in docs)

INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO: HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [3]:
# Testing the vector store
query = "What are the displayport details for the k800 series?"

docs = vectorstore.similarity_search(query)
# Retrieves the most semantically similar context document to the above query: 
docs[0].page_content

INFO: HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


'2.9 - DisplayPort\n\nThere are two full-size DisplayPorts on the Karbon 800 series. Both ports support DP 1.4 at 4K 60Hz\n\nand support MST (Multi Stream Topology). An MST hub can be used to support up to four\n\nindependent displays. Please refer to Intel documentation for additional Alder Lake-S display output\n\nspecifications: https://ark.intel.com/\n\n22'

In [4]:
from langchain_ollama import ChatOllama

# Load Llama3.1 model

model = ChatOllama(
    model='llama3.1:8b'
)


In [5]:
# Run LLM with context retrieval
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the chain

RAG_TEMPLATE = """
You are an assistant for tech support question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

retriever = vectorstore.as_retriever()

# The chain finds the most relevent context document using the vectorstore retriever, formats it,
# and inputs it into the template along with the user quesion. Then the model executues on the entire prompt, 
# and returns a response object. 

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
)

In [6]:
question = "What are the details for the K800 series of computers' CE compliance?"
response = chain.invoke(question)

INFO: HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO: HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [7]:
print(response.content)

The K800 series was evaluated for medical, IT equipment, automotive, maritime, and railway EMC standards as a class A device. It complies with the relevant IT equipment directives for the CE mark. Testing includes EN 55032, EN 55035, and other specified standards.


In [18]:
# Total run duration in seconds
total_dur = response.response_metadata['total_duration'] /  1e9
print(f'Total response duration: {total_dur:.2f} seconds')

Total response duration: 2.77 seconds


In [17]:
# Time to first token
print(f"Time to first token: {response.response_metadata['prompt_eval_duration'] / 1e9:,.2f} seconds")

Time to first token: 0.27 seconds


In [9]:
output_tokens_per_s = response.usage_metadata['output_tokens'] / response.response_metadata['eval_duration'] * 1e9
output_words_per_s = output_tokens_per_s / 1.3
output_words_per_m = output_words_per_s * 60

print(f'LLM output words per minute: {output_words_per_m:,.0f} WPM')

LLM output words per minute: 3,183 WPM
